In [3]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [4]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [5]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)

ChromeDriver version: 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [6]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [7]:
job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

#64ad5178b732b6c8d30ed4fa,64a7aef292a955755975d804,64a7ab5792a955755975d6a0

Enter keystone ids with comma.64d1d1b359aa371d9ed1a214
 Enter no of days for which data needs brand mapping.
1


('64d1d1b359aa371d9ed1a214', '')

In [8]:
query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\2162843895.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])


In [9]:
serp_details

,brand,title,sku,job_name,channel_id
0,PHILIPS,Philips HR2242/91 12 speed blender,1133782580,Liverpool r.no.565 Serp_job,97
1,CHEFMAN,Chefman 7.5 liter air fryer Rj38-sqpf-8t2p-w-mx,1133876100,Liverpool r.no.565 Serp_job,97
2,CHEFMAN,Chefman 3.5L RJ38-V2 3.5 Oil Fryer,1073762177,Liverpool r.no.565 Serp_job,97
3,CHEFMAN,Chefman Cheman RJ14-12-SQ Drip Coffee Maker,1096142541,Liverpool r.no.565 Serp_job,97
4,CHEFMAN,Chefman 7 liter air fryer RJ38-SQPF-8TW-MX,1126608329,Liverpool r.no.565 Serp_job,97
...,...,...,...,...,...
2774,CHEFMAN,Chefman RJ19-V2-SBP-Black Food Processor Set,1096538274,Liverpool r.no.565 Serp_job,97
2775,CHEFMAN,Chefman 10L RJ38-10-RDO-V2 Air Fryer,1099768696,Liverpool r.no.565 Serp_job,97
2776,CHEFMAN,Waffle Maker Chefman RJ04-4P 4 portions,1073762215,Liverpool r.no.565 Serp_job,97
2777,CHEFMAN,Chefman 7.4 liter air fryer + grill RJ38-AFG-7...,1127603215,Liverpool r.no.565 Serp_job,97


In [10]:
rawdf1=serp_details.drop_duplicates('sku')

df_ASIN = rawdf1[["sku"]]
df_ASIN.dropna(inplace=True)
df_ASIN.drop_duplicates(inplace=True)
asinlist = list(df_ASIN['sku'])
len(asinlist)

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\2610474507.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.dropna(inplace=True)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\2610474507.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.drop_duplicates(inplace=True)


817

In [11]:
query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(asinlist)} 
"""

pm_data = pd.read_sql_query(query_2, conn[1])
pm_data

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\3517360684.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pm_data = pd.read_sql_query(query_2, conn[1])


,channel_sku_id,country_code,brand_name,categories,category_id
0,1002761951,MX,LE CREUSET,None,None
1,1003055716,MX,Hamilton Beach,['Aparatos Eléctricos'],None
2,1003503565,MX,Hamilton,['Batidoras'],None
3,1004815994,MX,Oster,['Electrodomésticos Especializados'],None
4,1008134762,MX,Oster,['Aparatos Eléctricos'],None
...,...,...,...,...,...
821,4408489,MX,Sanelec,['Eléctricos y Multicontactos'],None
822,51037448,MX,Oster,['Licuadoras'],None
823,63740136,MX,OSTER,None,None
824,77504362,MX,Breville,['Aparatos Eléctricos'],None


In [12]:
mapped_df=pd.merge(rawdf1,pm_data, how='left', left_on=['sku'],right_on=['channel_sku_id'])
mapped_df

,brand,title,sku,job_name,channel_id,channel_sku_id,country_code,brand_name,categories,category_id
0,PHILIPS,Philips HR2242/91 12 speed blender,1133782580,Liverpool r.no.565 Serp_job,97,1133782580,MX,Philips,['Licuadoras'],None
1,CHEFMAN,Chefman 7.5 liter air fryer Rj38-sqpf-8t2p-w-mx,1133876100,Liverpool r.no.565 Serp_job,97,1133876100,MX,Chefman,['Freidoras'],None
2,CHEFMAN,Chefman 3.5L RJ38-V2 3.5 Oil Fryer,1073762177,Liverpool r.no.565 Serp_job,97,1073762177,MX,Chefman,['Freidoras de Aceite'],None
3,CHEFMAN,Chefman Cheman RJ14-12-SQ Drip Coffee Maker,1096142541,Liverpool r.no.565 Serp_job,97,1096142541,MX,Chefman,['Cafeteras'],None
4,CHEFMAN,Chefman 7 liter air fryer RJ38-SQPF-8TW-MX,1126608329,Liverpool r.no.565 Serp_job,97,1126608329,MX,Chefman,['Freidoras de Aire'],None
...,...,...,...,...,...,...,...,...,...,...
821,UFESA,Ufesa Malaga blender 10 speeds BS7020-MG-5291,1126387888,Liverpool r.no.565 Serp_job,97,1126387888,MX,Ufesa,['Licuadoras'],None
822,MAN,Man Ecoblender 3-speed blender LP-1010,1099235311,Liverpool r.no.565 Serp_job,97,1099235311,MX,Man,['Aparatos Eléctricos'],None
823,CHEFMAN,Waffle Maker Chefman RJ04-4P 4 portions,1073762215,Liverpool r.no.565 Serp_job,97,1073762215,MX,Chefman,['Cocinando Juntos'],None
824,CHEFMAN,Chefman 7.4 liter air fryer + grill RJ38-AFG-7...,1127603215,Liverpool r.no.565 Serp_job,97,1127603215,MX,Chefman,['Freidoras de Aire'],None


In [13]:
df2=mapped_df[mapped_df['categories'].isnull()][['channel_id','sku','title','brand_name']]
df2

,channel_id,sku,title,brand_name
17,97,1139996875,Chefman Chef iQ Electric Pressure Cooker,CHEFMAN
24,97,1106271492,Chefman RJ48 4 Liter Mini Refrigerator,CHEFMAN
36,97,1099398678,Black + Decker BL-1000WG 10-Speed Blender,BLACK + DECKER
48,97,1063056428,Hamilton Beach 2 Speed Motorized Blender,HAMILTON
54,97,1098999627,Kitchenaid K150 3 speed blender,KITCHENAID
...,...,...,...,...
767,97,1100369954,Avent electric breast pump,AVENT
774,97,1100530801,Electric epilator for legs Philips Series 8000...,PHILIPS
777,97,1135529369,Philips shaving set,PHILIPS
795,97,1138606178,Hamilton Beach 46895 Drip Coffee Maker,HAMILTON BEACH


In [17]:
asinlist = list(df2['sku'])
len(asinlist)

202

In [15]:
df=pd.DataFrame(columns=['channel_sku_id','categories'])
df

,channel_sku_id,categories


In [21]:
for i in tqdm(range(0,len(asinlist))):
    Url='https://www.liverpool.com.mx/tienda/pdp/-/'+asinlist[i]+'?skuid='+asinlist[i]
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument("--incognito")
#     driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)

    print('************************')
    print(i)
    print(Url)

    driver.get('https://www.liverpool.com.mx/tienda/pdp/lipstick-m.a.c-matte/1002183010?skuid=1014306036')
    time.sleep(2)
    bs=BeautifulSoup(driver.page_source, "html.parser")
    cat=[]
        
    try:
        aa=bs.find('ul',class_="m-breadcrumb-list")
        allli=aa.find_all('li')
        for eachli in allli:
            categ=eachli.find('a').text.strip()
            cat.append(categ)
        print(cat)
    except:
        print('not found')
    
    df1={'channel_sku_id': sku, 'categories': cat}
    df=df.append(df1, ignore_index = True)

  0%|                                                                                          | 0/202 [00:00<?, ?it/s]

************************
0
https://www.liverpool.com.mx/tienda/pdp/-/1139996875?skuid=1139996875


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\1117802599.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
  0%|▍                                                                                 | 1/202 [00:07<24:13,  7.23s/it]

['Home', 'Belleza', 'Marcas Destacadas', 'MAC']
************************
1
https://www.liverpool.com.mx/tienda/pdp/-/1106271492?skuid=1106271492


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\1117802599.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
  1%|▊                                                                                 | 2/202 [00:14<23:27,  7.04s/it]

...

['Home', 'Belleza', 'Marcas Destacadas', 'MAC']
************************
200
https://www.liverpool.com.mx/tienda/pdp/-/1138606178?skuid=1138606178


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\1117802599.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
100%|███████████████████████████████████████████████████████████████████████████████▌| 201/202 [22:41<00:06,  6.29s/it]

['Home', 'Belleza', 'Marcas Destacadas', 'MAC']
************************
201
https://www.liverpool.com.mx/tienda/pdp/-/1134529128?skuid=1134529128


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_155092\1117802599.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
100%|████████████████████████████████████████████████████████████████████████████████| 202/202 [22:47<00:00,  6.77s/it]

['Home', 'Belleza', 'Marcas Destacadas', 'MAC']


In [100]:
df

,index,brand,title,sku,job_name,channel_id,categories,sold_by,rating_count
0,0,OSTER,Oster 2-speed blender 2114087/BEST02-E00-13,1109553111,Liverpool r.no.565 Serp_job,97,[Licuadoras],STOREPOOL,
1,1,OSTER,Oster BLSTBPST-013 2-speed blender 2114099,1104553521,Liverpool r.no.565 Serp_job,97,[Licuadoras],,
2,2,OSTER,Oster blender BLST4655-013 3 speeds 2114081,1104552770,Liverpool r.no.565 Serp_job,97,[Licuadoras],,77 opiniones Escribe una opinión
3,3,OSTER,Oster Osterizer Cycle Blend Vintage 10-Speed B...,1109553081,Liverpool r.no.565 Serp_job,97,[Licuadoras],IBUSHAK,1 opinión Escribe una opinión
4,4,OSTER,Oster 2114101 2 speed blender,1104756146,Liverpool r.no.565 Serp_job,97,[Licuadoras],,37 opiniones Escribe una opinión
...,...,...,...,...,...,...,...,...,...
731,2071,AVENT,Set milk storage bags Avent SCF603/25,1044138782,Liverpool r.no.565 Serp_job,97,,,
732,2072,PHILIPS,Philips Series 5000 Shaver S5898/17,1135400528,Liverpool r.no.565 Serp_job,97,,,
733,2073,AVENT,Avent Formula Dispenser,1044138502,Liverpool r.no.565 Serp_job,97,,,
734,2074,AVENT,Avent Baby Bottles 9oz,1094071654,Liverpool r.no.565 Serp_job,97,,,


In [101]:
file='brand_mapping_liverpool {}.xlsx'.format(serp_details['job_name'][0])
file
final_df.to_excel(file)

'brand_mapping_liverpool Liverpool r.no.565 Serp_job .xlsx'

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','sambhav.jain@1digitalstack.ai','arihant.singh@1digitalstack.ai','kinshu@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s brand mapping of amazon. Please check brand mapping for following jobs.\n{}'.format(jobs_name)
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()